<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/04_text_classification_with_Transformer_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデルを使ったテキスト分類
* テキスト分類はBoWでも良い性能を出せることが多い。
  * 言語モデルを使って文書分類するときは・・・
  * BoW+SVMをベースラインとして設定して・・・
  * 言語モデルを使った場合と性能比較した方が良い。
* 今回はトランスフォーマ言語モデルを使ってみる。
  * パラメータ数は数億個オーダのもの。
  * パラメータ数が数十億個（数ビリオン）のものは、扱いがやや大変。

* ランタイムのタイプをGPUに設定しておく。

## インストール
* テキスト埋め込みにSentence Transformersというライブラリを使う。
  * https://sbert.net/

In [ ]:
!pip install -U sentence-transformers datasets

## 準備

In [ ]:
from transformers import set_seed

set_seed(0)

## データセット
* ライブドアニュースコーパスを使う。
* 前々回に作成したtraining/validation/testのsliceを使う。
  * https://github.com/tomonari-masada/course2024-nlp/blob/main/livedoor_ds.tar.gz

In [ ]:
!wget https://github.com/tomonari-masada/course2024-nlp/raw/refs/heads/main/livedoor_ds.tar.gz

In [ ]:
!tar zxvf livedoor_ds.tar.gz

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("livedoor_ds")

In [ ]:
ds

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

In [ ]:
ds["train"]["content"][0]

## テキストの埋め込み

###  埋め込みのための言語モデル
* 下のリーダボードから性能が良さそうなものを選ぶ。
  * https://huggingface.co/spaces/mteb/leaderboard
* ただし、日本語対応しているものを選ぶ。
* パラメータ数は数百millionのものを選ぶ。
  * 1桁上のパラメータ数だと、Google Colab無料版の場合、GPUのメモリが足りなくなる。
  * ただし、特別なことをすれば、可能（いずれ説明）。
* 新しいモデルはsentence transformerが対応していないことがある。
  * エラーが出たら、他の言語モデルに変える。


* ここではE5の多言語版を使ってみる。
  * https://arxiv.org/abs/2212.03533
  * https://arxiv.org/abs/2402.05672

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")

In [ ]:
model

* Transformerの部分

In [ ]:
list(model)[0]

* 言語モデルの本体
  * テキストを埋め込んでいる間に解説する。

In [ ]:
list(list(model)[0].modules())[1]

* 例えばword embeddingsの重み。

In [ ]:
list(list(model)[0].modules())[1].embeddings.word_embeddings.weight

### 埋め込みの実行
* 訓練セットの埋め込みは・・・
  * Google Colab無料版で10分弱。
  * iMac Apple M3で18分。
  * RTX4090で50秒(!)

In [ ]:
train_embeddings = model.encode(ds["train"]["content"], show_progress_bar=True)

* Google Driveの適当なディレクトリに保存する。

In [ ]:
import os
import numpy as np

data_dir = "/content/drive/MyDrive/2024courses/nlp/livedoor/gpu"

with open(os.path.join(data_dir, 'train_embeddings.npy'), 'wb') as f:
  np.save(f, train_embeddings)

* 検証セットとテストセットも埋め込む。

In [ ]:
validation_embeddings = model.encode(ds["validation"]["content"], show_progress_bar=True)
test_embeddings = model.encode(ds["test"]["content"], show_progress_bar=True)

In [ ]:
with open(os.path.join(data_dir, 'validation_embeddings.npy'), 'wb') as f:
  np.save(f, validation_embeddings)
with open(os.path.join(data_dir, 'test_embeddings.npy'), 'wb') as f:
  np.save(f, test_embeddings)

### 埋め込みベクトルの読み込み

In [ ]:
import os
import numpy as np

data_dir = "/content/drive/MyDrive/2024courses/nlp/livedoor/gpu"

with open(os.path.join(data_dir, 'train_embeddings.npy'), 'rb') as f:
  train_embeddings = np.load(f)
with open(os.path.join(data_dir, 'validation_embeddings.npy'), 'rb') as f:
  validation_embeddings = np.load(f)
with open(os.path.join(data_dir, 'test_embeddings.npy'), 'rb') as f:
  test_embeddings = np.load(f)

In [ ]:
train_embeddings.shape

## SVMによる分類

* 訓練セットと検証をセットを結合する。
  * 後で交差検証をするため。

In [ ]:
embeddings = np.concatenate([train_embeddings, validation_embeddings])
labels = np.array(ds["train"]["category"] + ds["validation"]["category"])

* 交差検証によりハイパーパラメータをチューニングする。

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)

for C in 10. ** np.arange(-1, 4):
  scores = []
  skf_split = skf.split(embeddings, labels)
  for train_index, val_index in skf_split:
    X_train = embeddings[train_index]
    clf = LinearSVC(C=C, dual=False, max_iter=1000, random_state=123)
    clf.fit(X_train, labels[train_index])
    X_val = embeddings[val_index]
    score = clf.score(X_val, labels[val_index])
    print(f"\t{score:.3f}", end=" ")
    scores.append(score)
  print(f"\nmean accuracy: {np.array(scores).mean():.3f}", end="")
  print(f" | C={C:.2e}")

* 選ばれた設定をテストセットで評価する。

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=10.0, dual=False, max_iter=1000, random_state=123)
clf.fit(embeddings, labels)
score = clf.score(test_embeddings, ds["test"]["category"])
print(f"{score:.3f}")

# 課題
* 作成済みの埋め込みベクトルを使って・・・
* 分類手法のほう（SVM以外でも構わない）だけをチューニングすることで・・・
* どのくらい分類性能を上げられるか、試行錯誤してみてください。
  * 今回の課題では、そんなに良い分類性能は出せないと思います。
  * パラメータ数が数億個の言語モデルであれば、ファインチューニングして使うのが、通常です。
  * その方法は、いずれ説明します。